# Depth series forecasting

## Setup

In [ ]:
import os
import datetime
import json
import csv
from collections import defaultdict
import IPython
from IPython.display import Image, display
from IPython.lib.display import Audio
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import keras
from keras import initializers
from sklearn.mixture import GaussianMixture
from sklearn.utils import shuffle
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
keras.utils.set_random_seed(100000)
tf.config.experimental.enable_op_determinism()
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.interpolate import interp1d

##  The multi-modal well logs dataset

In [ ]:
wells = pd.read_excel('wells 1-4.xlsx')
well_5 = pd.read_csv("well 5.csv")[['Depth', 'CGR', 'SGR', 'Facies']]
print(well_5.shape)
well_5.head()

In [ ]:
well_5.Facies.value_counts()

In [ ]:
plot_cols = ['CGR', 'SGR', 'Facies']
plot_features = well_5[plot_cols]
_ = plot_features.plot(subplots=True)

In [ ]:
well_5.describe().transpose()

### Data pre-processing

In [ ]:
well_1 = wells[wells['Well Name']==1][['Depth', 'CGR', 'SGR', 'Facies']]
well_2 = wells[wells['Well Name']==2][['Depth', 'CGR', 'SGR', 'Facies']]
well_3 = wells[wells['Well Name']==3][['Depth', 'CGR', 'SGR', 'Facies']]
well_4 = wells[wells['Well Name']==4][['Depth', 'CGR', 'SGR', 'Facies']]

def preprocess_well(df):
    df = df.copy()
    
    for col in ['CGR', 'SGR']:
        df[col] = np.log1p(df[col].clip(lower=0))
    
    scaler = StandardScaler()
    df[['CGR', 'SGR']] = scaler.fit_transform(df[['CGR', 'SGR']])
    
    df['TVD_increment'] = df['Depth'].diff().fillna(0)  
    
    df['CGR_depth_norm'] = df['CGR'] / (df['Depth'] + 1e-5)
    df['SGR_depth_norm'] = df['SGR'] / (df['Depth'] + 1e-5)
    
    df['dCGR_dDepth'] = df['CGR'].diff().fillna(0) / df['TVD_increment'].replace(0, np.nan)
    df['dSGR_dDepth'] = df['SGR'].diff().fillna(0) / df['TVD_increment'].replace(0, np.nan)
    df[['dCGR_dDepth', 'dSGR_dDepth']] = df[['dCGR_dDepth', 'dSGR_dDepth']].fillna(0)
    
    window = 5
    df['CGR_moving_avg'] = df['CGR'].rolling(window=window, min_periods=1).mean()
    df['CGR_moving_std'] = df['CGR'].rolling(window=window, min_periods=1).std().fillna(0)
    df['SGR_moving_avg'] = df['SGR'].rolling(window=window, min_periods=1).mean()
    df['SGR_moving_std'] = df['SGR'].rolling(window=window, min_periods=1).std().fillna(0)
    
    for col in ['CGR', 'SGR']:
        mask = df[col].isnull() | (np.abs(df[col] - df[col].median()) > 3 * df[col].std())
        if mask.any():
            interp_func = interp1d(df.loc[~mask, 'Depth'], df.loc[~mask, col], kind='linear', fill_value='extrapolate')
            df.loc[mask, col] = interp_func(df.loc[mask, 'Depth'])
    
    return df

well_1_pp = preprocess_well(well_1)
well_2_pp = preprocess_well(well_2)
well_3_pp = preprocess_well(well_3)
well_4_pp = preprocess_well(well_4)
well_5_pp = preprocess_well(well_5)

def align_wells(reference_df, wells_list):
    aligned_wells = []
    ref_depths = reference_df['Depth'].values
    for df in wells_list:
        interp_df = pd.DataFrame({'Depth': ref_depths})
        for col in df.columns:
            if col != 'Depth' and col != 'Facies':
                interp_func = interp1d(df['Depth'], df[col], kind='linear', bounds_error=False, fill_value='extrapolate')
                interp_df[col] = interp_func(ref_depths)
        if 'Facies' in df.columns:
            interp_df['Facies'] = reference_df['Facies'].values
        aligned_wells.append(interp_df)
    return aligned_wells

aligned_wells = align_wells(well_5_pp, [well_1_pp, well_2_pp, well_3_pp, well_4_pp])

merged_data = well_5_pp.copy()
for i, aw in enumerate(aligned_wells, start=1):
    suffix = f'_{i}'
    for col in aw.columns:
        if col not in ['Depth', 'Facies']:
            merged_data[col + suffix] = aw[col].values

print(merged_data.shape)
merged_data.head()


## Raw data visualisation

In [ ]:
formations_dict= {}
with open('Formation.csv', 'r') as file:
    next(file) 
    for row in csv.DictReader(file, fieldnames=['Formation', 'Top', 'Bottom']):
        formations_dict[row['Formation']]=[float(row['Top']), float(row['Bottom'])]
formations_dict
formation_midpoints = []
for key, value in formations_dict.items():
    formation_midpoints.append(value[0] + (value[1]-value[0])/2)
lithology_numbers = {
                     1: {'lith':'argiLs', 'lith_num':1, 'hatch':'...', 'color':'#0000FF'},
                     2: {'lith':'Sh', 'lith_num':2, 'hatch':'--', 'color':'#565051'},
                     3: {'lith':'chkLs', 'lith_num':3, 'hatch':'//', 'color':'#33caff'},
                     4: {'lith':'Ls', 'lith_num':4, 'hatch':'o', 'color':'#00FFFF'},
                    }

In [ ]:
y = [0, 1]
x = [1, 1]
fig, axes = plt.subplots(ncols=1,nrows=4, sharex=True, sharey=True,
                         figsize=(1,3.3), subplot_kw={'xticks': [], 'yticks': []})
for ax, key in zip(axes.flat, lithology_numbers.keys()):
    ax.plot(x, y)
    ax.fill_betweenx(y, 0, 1, facecolor=lithology_numbers[key]['color'], hatch=lithology_numbers[key]['hatch'])
    ax.set_xlim(0, 0.1)
    ax.set_ylim(0, 1)
    ax.set_title(str(lithology_numbers[key]['lith']))   
plt.tight_layout()
plt.subplots_adjust(wspace = 0.35)
plt.savefig("facies_guide.tiff", dpi=300)   
plt.show()

In [ ]:
formation_numbers = {
                     1: {'lith':'Gurpi', 'lith_num':1, 'hatch':'', 'color':'#A9E2F3'},
                     2: {'lith':'Ilam', 'lith_num':2, 'hatch':'', 'color':'#585858'},
                     3: {'lith':'Lafan', 'lith_num':3, 'hatch':'', 'color':'#F7FE2E'},
                     4: {'lith':'Sarvak', 'lith_num':4, 'hatch':'', 'color':'#D8D8D8'},
                     5: {'lith':'Kazhdumi', 'lith_num':5, 'hatch':'', 'color':'#81F781'},}
formation_numbers[4]['color']
df_lith = pd.DataFrame.from_dict(formation_numbers, orient='index')
df_lith.index.name = 'Formation'
df_lith
y = [0, 1]
x = [1, 1]
fig, axes = plt.subplots(ncols=1,nrows=5, sharex=True, sharey=True,
                         figsize=(1,4), subplot_kw={'xticks': [], 'yticks': []})
for ax, key in zip(axes.flat, formation_numbers.keys()):
    ax.plot(x, y)
    ax.fill_betweenx(y, 0, 1, facecolor=formation_numbers[key]['color'], hatch=formation_numbers[key]['hatch'])
    ax.set_xlim(0, 0.1)
    ax.set_ylim(0, 1)
    ax.set_title(str(formation_numbers[key]['lith'])) 
plt.tight_layout()
plt.subplots_adjust(wspace = 0.35)
plt.savefig("formation_guide.tiff", dpi=300)
plt.show()

In [ ]:
formations = {"Gurpi":[2350.618, 2659.99],
              "Ilam": [2660.142, 2757.938],
              "Lafan": [2758.135, 2768.956],
              "Sarvak": [2769.108, 3398.977]}
zone_colours = ['#A9E2F3',
       '#585858','#F7FE2E','#D8D8D8', '#00FFFF']
def makeplot(well, top_depth, bottom_depth, formations):
    fig, ax = plt.subplots(figsize=(3.8, 8))

    ax1 = plt.subplot2grid((1, 3), (0, 0), rowspan=1, colspan = 1)
    
    ax2 = plt.subplot2grid((1, 3), (0, 1), rowspan=1, colspan = 1, sharey = ax1)
    ax10 = ax1.twiny()
    ax10.xaxis.set_visible(False)
    ax1.plot(well["CGR"], well['Depth'], color = "#FF0000", linewidth = 0.8)
    ax1.set_xlabel("GR (GAPI)")
    ax1.xaxis.label.set_color("#FF0000")
    ax1.set_xlim(0, 150)
    ax1.set_ylabel("TVD (m)")
    ax1.tick_params(axis='x', colors="#FF0000")
    ax1.spines["top"].set_edgecolor("#FF0000")
    ax1.title.set_color('#FF0000')
    ax1.set_xticks([0, 75, 150])
    left_col_value = 0
    right_col_value = 150
    span = abs(left_col_value - right_col_value)
    cmap = plt.get_cmap('hot_r')
    color_index = np.arange(left_col_value, right_col_value, span / 100)
    for index in sorted(color_index):
        index_value = (index - left_col_value)/span
        color = cmap(index_value) 
    ax2.plot(well["LITHOLOGY"], well['Depth'], color = "black", linewidth = 0.5)
    ax2.set_xlabel("GML")
    ax2.set_xlim(0, 1)
    ax2.xaxis.label.set_color("black")
    ax2.tick_params(axis='x', colors="black")
    ax2.spines["top"].set_edgecolor("black")
    for key in lithology_numbers.keys():
        color = lithology_numbers[key]['color']
        hatch = lithology_numbers[key]['hatch']
        ax2.fill_betweenx(well['Depth'], 0, well['LITHOLOGY'], where=(well['LITHOLOGY']==key),
                         facecolor=color, hatch=hatch)
    ax2.set_xticks([])
    for ax in [ax1, ax2]:
        ax.set_ylim(bottom_depth, top_depth)
        ax.grid(which='major', color='lightgrey', linestyle='-')
        ax.xaxis.set_ticks_position("top")
        ax.xaxis.set_label_position("top")
        ax.spines["top"].set_position(("axes", 1.01))
    for ax in [ax1]:
        for depth, colour in zip(formations.values(), zone_colours):
            ax.axhspan(depth[0], depth[1], color=colour, alpha=0.5)
    for ax in [ ax2]:
        plt.setp(ax.get_yticklabels(), visible = False)
    for label, formation_mid in zip(formations_dict.keys(),
                                    formation_midpoints):
        ax1.text(5, formation_mid, label, rotation=0,
                verticalalignment='center', fontweight='bold',
                fontsize='large')
    plt.tight_layout()
    fig.subplots_adjust(wspace = 0.22)
    fig.savefig("total.tiff", dpi=300)
df = 'df_raw.csv'
df = pd.read_csv(df)
makeplot(df, 2350, 3398, formations)

### Split the data

In [ ]:
data = well_5.merge(well_1, on='Depth', suffixes = (None,'_1'))
print(data.shape)
data = data.merge(well_2, on='Depth', suffixes = (None,'_2'))
print(data.shape)
data = data.merge(well_3, on='Depth', suffixes = (None,'_3'))
print(data.shape)
data = data.merge(well_4, on='Depth', suffixes = (None,'_4'))
print(data.shape)
column_indices = {name: i for i, name in enumerate(data.columns)}
n = len(data)
train_df = data[data.Depth <= 2779.9284]
val_df = data[(data.Depth > 2779.9284) & (data.Depth <= 2900.)]
test_df = data[data.Depth > 2900.]
num_features = data.shape[1]
train_df.shape, val_df.shape, test_df.shape

In [ ]:
test_df

In [ ]:
test_f=test_df['Facies']

In [ ]:
test_f

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

In [ ]:
df_std = (data - train_mean) / train_std
df_std = df_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(data.keys(), rotation=90)

## Data windowing

In [ ]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
                 train_df, val_df, test_df,
                 label_columns=None):
        # Raw datasets
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift
        self.total_window_size = input_width + shift
        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]
        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]
    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)
            inputs = tf.stack(
                [inputs[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)
        mean = tf.reduce_mean(inputs, axis=1, keepdims=True)
        std = tf.math.reduce_std(inputs, axis=1, keepdims=True) + 1e-6  
        inputs = (inputs - mean) / std
        return inputs, labels
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32)
        ds = ds.map(self.split_window)
        return ds
    @property
    def train(self):
        return self.make_dataset(self.train_df)
    @property
    def val(self):
        return self.make_dataset(self.val_df)
    @property
    def test(self):
        return self.make_dataset(self.test_df)

In [ ]:
w1 = WindowGenerator(
    input_width=24,
    label_width=1,
    shift=24,
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    label_columns=['CGR', 'SGR', 'Facies'])
w1

In [ ]:
w2 = WindowGenerator(
    input_width=8,
    label_width=1,
    shift=1,
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    label_columns=['CGR', 'SGR', 'Facies'])
w2

In [ ]:
def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
        labels = tf.stack(
            [labels[:, :, self.column_indices[name]] for name in self.label_columns],
            axis=-1)
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])
    return inputs, labels
WindowGenerator.split_window = split_window

In [ ]:
example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])
example_inputs, example_labels = w2.split_window(example_window)
print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

In [ ]:
w2.example = example_inputs, example_labels

In [ ]:
def plot(self, plot_col = ['CGR', 'SGR', 'Facies'], model=None, max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  cols = [self.column_indices[col] for col in plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plot_col_index = self.column_indices[plot_col[n]]
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col[n]}')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)
    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col[n], None)
    else:
      label_col_index = plot_col_index
    if label_col_index is None:
      continue
    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)
    plt.legend()
    if plot_col[n] == "Facies":
      plt.ylim(bottom=0, top=5)
    else:
      plt.ylim(bottom=-10, top=100)
  plt.xlabel('Depth')
WindowGenerator.plot = plot

In [ ]:
def loss_plot(history):
  fig, ax = plt.subplots(1, 2)
  fig.set_size_inches(10, 4)
  ax[0].plot(history['loss'], label = 'loss')
  ax[0].plot(history['val_loss'], label = 'Val_loss')
  ax[0].legend()
  ax[0].set_xlabel('EPOCHS')
  ax[0].set_ylabel("Loss")
  ax[1].plot(history["mean_absolute_percentage_error"], label = "MAPE")
  ax[1].plot(history["val_mean_absolute_percentage_error"], label = 'Val-MAPE')
  ax[1].legend()
  ax[1].set_xlabel('EPOCHS')
  ax[1].set_ylabel("MAPE")
  plt.plot()

In [ ]:
w2.plot(plot_col = ['CGR', 'SGR', 'Facies'])

In [ ]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=False,
      batch_size=32,)
  ds = ds.map(self.split_window)
  return ds
WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df)
@property
def val(self):
  return self.make_dataset(self.val_df)
@property
def test(self):
  return self.make_dataset(self.test_df)
@property
def example(self):
  result = getattr(self, '_example', None)
  if result is None:
    result = next(iter(self.train))
    self._example = result
  return result
WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [ ]:
w2.train.element_spec

Iterating over a `Dataset` yields concrete batches:

In [ ]:
for example_inputs, example_labels in w2.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

## Single step models

In [ ]:
single_step_window = WindowGenerator(
    input_width=1,
    label_width=1,
    shift=1,
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    label_columns=['CGR', 'SGR', 'Facies'])
single_step_window

In [ ]:
for example_inputs, example_labels in single_step_window.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

### Normalize Layer

In [ ]:
class norm_layer(tf.keras.layers.Layer):
  def __init__(self, M, S):
    super(norm_layer, self).__init__()
    self.mean = M
    self.std  = S
  def call(self, inp):
    return (inp - self.mean) / self.std

In [ ]:
MAX_EPOCHS = 200
class HybridCLFLoss(tf.keras.losses.Loss):
    def __init__(self, model, lambda_l1=1e-6, lambda_dr=1e-5, name="hybrid_clf_loss"):
        super().__init__(name=name)
        self.model = model
        self.lambda_l1 = lambda_l1
        self.lambda_dr = lambda_dr
    def call(self, y_true, y_pred, sample_weight=None):
        if sample_weight is not None:
            weighted_error = sample_weight * tf.square(y_true - y_pred)
            mse = tf.reduce_mean(weighted_error)
        else:
            mse = tf.reduce_mean(tf.square(y_true - y_pred))
        mse = tf.where(tf.math.is_nan(mse), tf.zeros_like(mse), mse)
        reg_loss = tf.add_n([
            tf.reduce_sum(tf.abs(layer.kernel))
            for layer in self.model.layers if isinstance(layer, tf.keras.layers.Dense)
        ])
        reg_loss = tf.where(tf.math.is_nan(reg_loss), tf.zeros_like(reg_loss), reg_loss)
        if len(y_pred.shape) >= 2 and tf.shape(y_pred)[1] > 1:
            diff = y_pred[:, 1:] - y_pred[:, :-1]
            dr_loss = tf.reduce_mean(tf.square(diff))
        else:
            dr_loss = 0.0
        dr_loss = tf.where(tf.math.is_nan(dr_loss), tf.zeros_like(dr_loss), dr_loss)
        return mse + self.lambda_l1 * reg_loss + self.lambda_dr * dr_loss
class WeightedLossWrapper(tf.keras.losses.Loss):
    def __init__(self, base_loss, sample_weights):
        super().__init__()
        self.base_loss = base_loss
        self.sample_weights = tf.convert_to_tensor(sample_weights, dtype=tf.float32)
    def call(self, y_true, y_pred):
        return self.base_loss(y_true, y_pred, sample_weight=self.sample_weights)
def compile_and_fit(model, window, patience=50, lambda_l1=1e-6, lambda_dr=1e-5, sample_weights=None):
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=patience,
        mode='min',
        restore_best_weights=True)
    base_loss = HybridCLFLoss(model=model, lambda_l1=lambda_l1, lambda_dr=lambda_dr)
    if sample_weights is not None:
        loss_fn = WeightedLossWrapper(base_loss, sample_weights)
    else:
        loss_fn = lambda y_true, y_pred: base_loss(y_true, y_pred)
    model.compile(
        loss=loss_fn,
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
        metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])
    history = model.fit(
        window.train,
        validation_data=window.val,
        epochs=MAX_EPOCHS,
        callbacks=[early_stopping])
    return history

In [ ]:
performance = {}
val_performance = {}

In [ ]:
wide_window = WindowGenerator(
    input_width=24,
    label_width=24,
    shift=1,
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    label_columns=['CGR', 'SGR', 'Facies'])
wide_window

### MLP

In [ ]:
D_s = shuffle(train_df, random_state=42)
D_s = D_s.drop(columns=["Depth"])
D_t = val_df.drop(columns=["Depth"])
mlp_model = tf.keras.Sequential([
    norm_layer(train_mean, train_std),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=3)])
compile_and_fit(mlp_model, single_step_window)
mlp_model.save_weights("pretrained_weights.h5")  
mlp_model.load_weights("pretrained_weights.h5")
compile_and_fit(mlp_model, single_step_window)   
gmm_data = train_df.drop(columns=["Depth"])
gmm = GaussianMixture(n_components=5, covariance_type='full', random_state=42)
gmm.fit(gmm_data)
num_samples = len(gmm_data)
synthetic_samples, _ = gmm.sample(num_samples)
D_syn = pd.DataFrame(synthetic_samples, columns=gmm_data.columns)
D_o = train_df.drop(columns=["Depth"])
D_c = pd.concat([D_o, D_syn], ignore_index=True)
mlp_model.load_weights("pretrained_weights.h5")
history = compile_and_fit(mlp_model, single_step_window)
val_performance['Dense'] = mlp_model.evaluate(single_step_window.val, verbose=0)
performance['Dense'] = mlp_model.evaluate(single_step_window.test, verbose=0)

In [ ]:
loss_plot(history.history)

In [ ]:
wide_window.plot(model = mlp_model, max_subplots=3)

### LSTM

In [ ]:
D_s = shuffle(train_df, random_state=42).drop(columns=["Depth"])
D_t = val_df.drop(columns=["Depth"])
lstm_model = tf.keras.Sequential([
    norm_layer(train_mean, train_std),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(128, activation='relu'), 
    tf.keras.layers.Dense(3),])
_ = lstm_model(single_step_window.example[0])  
compile_and_fit(lstm_model, single_step_window)
lstm_model.save_weights("lstm_pretrained_weights.h5") 
lstm_model.load_weights("lstm_pretrained_weights.h5")
compile_and_fit(lstm_model, single_step_window)
gmm_data = train_df.drop(columns=["Depth"])
gmm = GaussianMixture(n_components=5, covariance_type='full', random_state=42)
gmm.fit(gmm_data)
synthetic_samples, _ = gmm.sample(len(gmm_data))
D_syn = pd.DataFrame(synthetic_samples, columns=gmm_data.columns)
D_o = train_df.drop(columns=["Depth"])
D_c = pd.concat([D_o, D_syn], ignore_index=True)
lstm_model.load_weights("lstm_pretrained_weights.h5")
compile_and_fit(lstm_model, single_step_window)
val_performance['LSTM'] = lstm_model.evaluate(single_step_window.val, verbose=0)
performance['LSTM'] = lstm_model.evaluate(single_step_window.test, verbose=0)
loss_plot(history.history)
wide_window.plot(model=lstm_model, max_subplots=3)

## Transformer

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="gelu"),
            layers.Dense(embed_dim),])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
class PositionalEncoding(layers.Layer):
    def __init__(self, position, d_model):
        super().__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)
    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles
    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)
    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]
def create_transformer_model(input_shape, output_dim, num_layers=3, d_model=128, num_heads=8, ff_dim=256, dropout=0.1):
    inputs = layers.Input(shape=input_shape)
    x = norm_layer(train_mean, train_std)(inputs)
    x = layers.Dense(d_model)(x)
    x = PositionalEncoding(input_shape[0], d_model)(x)
    for _ in range(num_layers):
        x = TransformerBlock(d_model, num_heads, ff_dim, dropout)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(output_dim)(x)
    outputs = layers.Reshape([1, output_dim])(outputs)
    return tf.keras.Model(inputs=inputs, outputs=outputs)
D_s = shuffle(train_df, random_state=42).drop(columns=["Depth"])
D_t = val_df.drop(columns=["Depth"])
input_shape = single_step_window.example[0].shape[1:]  
output_dim = 3
transformer_model = create_transformer_model(input_shape, output_dim)
_ = transformer_model(single_step_window.example[0])  
compile_and_fit(transformer_model, single_step_window)
transformer_model.save_weights("transformer_pretrained_weights.h5")
transformer_model.load_weights("transformer_pretrained_weights.h5")
compile_and_fit(transformer_model, single_step_window)
gmm_data = train_df.drop(columns=["Depth"])
gmm = GaussianMixture(n_components=5, covariance_type='full', random_state=42)
gmm.fit(gmm_data)
synthetic_samples, _ = gmm.sample(len(gmm_data))
D_syn = pd.DataFrame(synthetic_samples, columns=gmm_data.columns)
D_o = train_df.drop(columns=["Depth"])
D_c = pd.concat([D_o, D_syn], ignore_index=True)
transformer_model.load_weights("transformer_pretrained_weights.h5")
compile_and_fit(transformer_model, single_step_window)
val_performance['transformer_model'] = transformer_model.evaluate(single_step_window.val, verbose=0)
performance['transformer_model'] = transformer_model.evaluate(single_step_window.test, verbose=0)
transformer_model.summary()
loss_plot(history.history)
single_step_window.plot(model=transformer_model, max_subplots=3)

### Performance

In [ ]:
x = np.arange(len(performance))
width = 0.3
metric_name = 'mean_absolute_percentage_error'
metric_index = lstm_model.metrics_names.index('mean_absolute_percentage_error')
val_mae = [v[metric_index] for v in val_performance.values()]
test_mae = [v[metric_index] for v in performance.values()]
plt.ylabel('mean_absolute_percentage_error')
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=performance.keys(),
           rotation=45)
_ = plt.legend()

In [ ]:
for name, value in performance.items():
  print(f'{name:10}: {value[1]:0.4f}')

## Multi-step models

In [ ]:
multi_window = WindowGenerator(
    input_width=8,
    label_width=1,
    shift=1,
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    label_columns=['CGR', 'SGR', 'Facies'])
multi_window

In [ ]:
val_performance = {}
performance = {}

## MLP


In [ ]:
D_s = shuffle(train_df, random_state=42).drop(columns=["Depth"])
D_t = val_df.drop(columns=["Depth"])

# Step 2: Define and pre-train the MLP model on D_s
multi_mlp_model = tf.keras.Sequential([
    norm_layer(train_mean, train_std),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3),
    tf.keras.layers.Reshape([1, -1]),])
_ = multi_mlp_model(multi_window.example[0])  
compile_and_fit(multi_mlp_model, multi_window)
multi_mlp_model.save_weights("multi_mlp_pretrained_weights.h5")  # Save θ_p
multi_mlp_model.load_weights("multi_mlp_pretrained_weights.h5")
compile_and_fit(multi_mlp_model, multi_window)
gmm_data = train_df.drop(columns=["Depth"])
gmm = GaussianMixture(n_components=5, covariance_type='full', random_state=42)
gmm.fit(gmm_data)
synthetic_samples, _ = gmm.sample(len(gmm_data))
D_syn = pd.DataFrame(synthetic_samples, columns=gmm_data.columns)
D_o = train_df.drop(columns=["Depth"])
D_c = pd.concat([D_o, D_syn], ignore_index=True)
multi_mlp_model.load_weights("multi_mlp_pretrained_weights.h5")
compile_and_fit(multi_mlp_model, multi_window)
val_performance['mlp'] = multi_mlp_model.evaluate(multi_window.val, verbose=0)
performance['mlp'] = multi_mlp_model.evaluate(multi_window.test, verbose=0)
multi_mlp_model.summary()
loss_plot(history.history)
multi_window.plot(model=multi_mlp_model, max_subplots=3)

## LSTM

In [ ]:
D_s = shuffle(train_df, random_state=42).drop(columns=["Depth"])
D_t = val_df.drop(columns=["Depth"])
multi_lstm_model = tf.keras.Sequential([
    norm_layer(train_mean, train_std),
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3),
    tf.keras.layers.Reshape([1, -1]),])
_ = multi_lstm_model(multi_window.example[0])
compile_and_fit(multi_lstm_model, multi_window)
multi_lstm_model.save_weights("multi_lstm_pretrained_weights.h5") 
multi_lstm_model.load_weights("multi_lstm_pretrained_weights.h5")
compile_and_fit(multi_lstm_model, multi_window)
gmm_data = train_df.drop(columns=["Depth"])
gmm = GaussianMixture(n_components=5, covariance_type='full', random_state=42)
gmm.fit(gmm_data)
synthetic_samples, _ = gmm.sample(len(gmm_data))
D_syn = pd.DataFrame(synthetic_samples, columns=gmm_data.columns)
D_o = train_df.drop(columns=["Depth"])
D_c = pd.concat([D_o, D_syn], ignore_index=True)
multi_lstm_model.load_weights("multi_lstm_pretrained_weights.h5")
compile_and_fit(multi_lstm_model, multi_window)
val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val, verbose=0)
performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
multi_lstm_model.summary()
loss_plot(history.history)
multi_window.plot(model=multi_lstm_model, max_subplots=2)

## Transformer

In [ ]:
D_s = shuffle(train_df, random_state=42).drop(columns=["Depth"])
D_t = val_df.drop(columns=["Depth"])
input_shape = multi_window.example[0].shape[1:]  
output_dim = 3  
multi_transformer_model = create_transformer_model(input_shape, output_dim)
_ = multi_transformer_model(multi_window.example[0])  
compile_and_fit(multi_transformer_model, multi_window)
multi_transformer_model.save_weights("multi_transformer_pretrained_weights.h5")  
multi_transformer_model.load_weights("multi_transformer_pretrained_weights.h5")
compile_and_fit(multi_transformer_model, multi_window)
gmm_data = train_df.drop(columns=["Depth"])
gmm = GaussianMixture(n_components=5, covariance_type='full', random_state=42)
gmm.fit(gmm_data)
synthetic_samples, _ = gmm.sample(len(gmm_data))
D_syn = pd.DataFrame(synthetic_samples, columns=gmm_data.columns)
D_o = train_df.drop(columns=["Depth"])
D_c = pd.concat([D_o, D_syn], ignore_index=True)
multi_transformer_model.load_weights("multi_transformer_pretrained_weights.h5")
compile_and_fit(multi_transformer_model, multi_window)
val_performance['Transformer'] = multi_transformer_model.evaluate(multi_window.val, verbose=0)
performance['Transformer'] = multi_transformer_model.evaluate(multi_window.test, verbose=0)
multi_transformer_model.summary()
loss_plot(history.history)
multi_window.plot(model=multi_transformer_model, max_subplots=2)
def plot_model_structure(model, filename="model_structure.png", show=True):
    plot_model(model, to_file=filename, show_shapes=True, show_layer_names=True, 
               rankdir='TB', expand_nested=True, dpi=300)
    img = plt.imread(filename)
    fig = plt.figure(figsize=(12, 12), dpi=300)
    plt.imshow(img)
    plt.axis('off')
    plt.savefig(filename, format='png', dpi=300, bbox_inches='tight')
    if show:
        display(Image(filename))
    plt.close(fig)
plot_model_structure(multi_transformer_model, filename="transformer_model_structure.png")

In [ ]:
x = np.arange(len(performance))
width = 0.3
metric_name = 'mean_absolute_percentage_error'
metric_index = multi_lstm_model.metrics_names.index(metric_name)
val_mae = [v[metric_index] for v in val_performance.values()]
test_mae = [v[metric_index] for v in performance.values()]
plt.ylabel(metric_name)
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=performance.keys(), rotation=45)
_ = plt.legend()

In [ ]:
for name, value in performance.items():
    print(f'{name:15}: {value[metric_index]:0.4f}')

# Plot Results

In [ ]:
train = data[data.Depth < 2779.9284]
test = data[data.Depth > 2779.9284]

In [ ]:
data.rename(columns={'Facies':'LITHOLOGY'}, inplace=True)
data.head()

In [ ]:
data = data.iloc[:, :4]
data.head()

In [ ]:
formations_dict= {}

In [ ]:
with open('Formation.csv', 'r') as file:
    next(file)
    for row in csv.DictReader(file, fieldnames=['Formation', 'Top', 'Bottom']):
        formations_dict[row['Formation']]=[float(row['Top']), float(row['Bottom'])]

In [ ]:
formations_dict

In [ ]:
formation_midpoints = []
for key, value in formations_dict.items():
    formation_midpoints.append(value[0] + (value[1]-value[0])/2)

In [ ]:
lithology_numbers = {
                     1: {'lith':'argiLs', 'lith_num':1, 'hatch':'--|/', 'color':'#0000FF'},
                     2: {'lith':'Sh', 'lith_num':2, 'hatch':'--', 'color':'#565051'},
                     3: {'lith':'chkLs', 'lith_num':3, 'hatch':'.|.--', 'color':'#33caff'},
                     4: {'lith':'Ls', 'lith_num':4, 'hatch':'--|', 'color':'#00FFFF'}}

In [ ]:
formations_dict= {}
with open('Formation.csv', 'r') as file:
    next(file) #skip header row
    for row in csv.DictReader(file, fieldnames=['Formation', 'Top', 'Bottom']):
        formations_dict[row['Formation']]=[float(row['Top']), float(row['Bottom'])]
formations_dict
formation_midpoints = []
for key, value in formations_dict.items():
    formation_midpoints.append(value[0] + (value[1]-value[0])/2)
lithology_numbers = {
                     1: {'lith':'argiLs', 'lith_num':1, 'hatch':'...', 'color':'#0000FF'},
                     2: {'lith':'Sh', 'lith_num':2, 'hatch':'--', 'color':'#565051'},
                     3: {'lith':'chkLs', 'lith_num':3, 'hatch':'//', 'color':'#33caff'},
                     4: {'lith':'Ls', 'lith_num':4, 'hatch':'o', 'color':'#00FFFF'}}

In [ ]:
y = [0, 1]
x = [1, 1]

fig, axes = plt.subplots(ncols=1,nrows=4, sharex=True, sharey=True,
                         figsize=(1,3.3), subplot_kw={'xticks': [], 'yticks': []})
for ax, key in zip(axes.flat, lithology_numbers.keys()):
    ax.plot(x, y)
    ax.fill_betweenx(y, 0, 1, facecolor=lithology_numbers[key]['color'], hatch=lithology_numbers[key]['hatch'])
    ax.set_xlim(0, 0.1)
    ax.set_ylim(0, 1)
    ax.set_title(str(lithology_numbers[key]['lith']))  
plt.tight_layout()
plt.subplots_adjust(wspace = 0.35)
plt.savefig("facies_guide.tiff", dpi=300) 
plt.show()

In [ ]:
formation_numbers = {
                     1: {'lith':'Gurpi', 'lith_num':1, 'hatch':'', 'color':'#A9E2F3'},
                     2: {'lith':'Ilam', 'lith_num':2, 'hatch':'', 'color':'#585858'},
                     3: {'lith':'Lafan', 'lith_num':3, 'hatch':'', 'color':'#F7FE2E'},
                     4: {'lith':'Sarvak', 'lith_num':4, 'hatch':'', 'color':'#D8D8D8'},
                     5: {'lith':'Kazhdumi', 'lith_num':5, 'hatch':'', 'color':'#81F781'}}
formation_numbers[4]['color']
df_lith = pd.DataFrame.from_dict(formation_numbers, orient='index')
df_lith.index.name = 'Formation'
df_lith
y = [0, 1]
x = [1, 1]
fig, axes = plt.subplots(ncols=1,nrows=5, sharex=True, sharey=True,
                         figsize=(1,4), subplot_kw={'xticks': [], 'yticks': []})
for ax, key in zip(axes.flat, formation_numbers.keys()):
    ax.plot(x, y)
    ax.fill_betweenx(y, 0, 1, facecolor=formation_numbers[key]['color'], hatch=formation_numbers[key]['hatch'])
    ax.set_xlim(0, 0.1)
    ax.set_ylim(0, 1)
    ax.set_title(str(formation_numbers[key]['lith'])) 
plt.tight_layout()
plt.subplots_adjust(wspace = 0.35)
plt.savefig("formation_guide.tiff", dpi=300)
plt.show()

In [ ]:
formations = {"Gurpi":[2350.618, 2659.99],
              "Ilam": [2660.142, 2757.938],
              "Lafan": [2758.135, 2768.956],
              "Sarvak": [2769.108, 3398.977]}

In [ ]:
zone_colours = ['#A9E2F3',
       '#585858','#F7FE2E','#D8D8D8', '#00FFFF']
def makeplot(well, top_depth, bottom_depth, formations):
    fig, ax = plt.subplots(figsize=(3.8, 8))
    ax1 = plt.subplot2grid((1, 3), (0, 0), rowspan=1, colspan = 1)
    ax12 = ax1.twiny()
    ax2 = plt.subplot2grid((1, 3), (0, 1), rowspan=1, colspan = 1, sharey = ax1)
    ax3 = plt.subplot2grid((1, 3), (0, 2), rowspan=1, colspan = 1, sharey = ax1)

    ax10 = ax1.twiny()
    ax10.xaxis.set_visible(False)
    ax1.plot(well["val CGR"], well['Depth'], color = "#088A08", linewidth = 0.5)
    ax1.set_xlabel("S-GR (GAPI)")
    ax1.xaxis.label.set_color("#088A08")
    ax1.set_xlim(0, 150)
    ax1.set_ylabel("TVD (m)")
    ax1.tick_params(axis='x', colors="#088A08")
    ax1.spines["top"].set_edgecolor("#088A08")
    ax1.title.set_color('#088A08')
    ax1.set_xticks([0, 75, 150])
    left_col_value = 0
    right_col_value = 150
    span = abs(left_col_value - right_col_value)
    cmap = plt.get_cmap('hot_r')
    color_index = np.arange(left_col_value, right_col_value, span / 100)
    for index in sorted(color_index):
        index_value = (index - left_col_value)/span
        color = cmap(index_value) 
    ax12.plot(well["CGR"], well['Depth'], color = "#FF0000", linewidth = 0.5)
    ax12.set_xlabel('GR (GAPI)')
    ax12.xaxis.label.set_color("#FF0000")
    ax12.set_xlim(0, 150)
    ax12.tick_params(axis='x', colors="#FF0000")
    ax12.spines["top"].set_position(("axes", 1.075))
    ax12.spines["top"].set_visible(True)
    ax12.spines["top"].set_edgecolor("#FF0000")
    ax12.set_xticks([0, 75, 150])
    ax2.plot(well["LITHOLOGY"], well['Depth'], color = "black", linewidth = 0.5)
    ax2.set_xlabel("GML")
    ax2.set_xlim(0, 1)
    ax2.xaxis.label.set_color("black")
    ax2.tick_params(axis='x', colors="black")
    ax2.spines["top"].set_edgecolor("black")
    for key in lithology_numbers.keys():
        color = lithology_numbers[key]['color']
        hatch = lithology_numbers[key]['hatch']
        ax2.fill_betweenx(well['Depth'], 0, well['LITHOLOGY'], where=(well['LITHOLOGY']==key),
                         facecolor=color, hatch=hatch)
    ax2.set_xticks([])
    ax3.plot(well["val LITHOLOGY"], well['Depth'], color = "black", linewidth = 0.5)
    ax3.set_xlabel("S-GML")
    ax3.set_xlim(0, 1)
    ax3.xaxis.label.set_color("black")
    ax3.tick_params(axis='x', colors="black")
    ax3.spines["top"].set_edgecolor("black")

    for key in lithology_numbers.keys():
        color = lithology_numbers[key]['color']
        hatch = lithology_numbers[key]['hatch']
        ax3.fill_betweenx(well['Depth'], 0, well['val LITHOLOGY'], where=(well['val LITHOLOGY']==key),
                         facecolor=color, hatch=hatch)
    ax3.set_xticks([])
    for ax in [ax1, ax2, ax3]:
        ax.set_ylim(bottom_depth, top_depth)
        ax.grid(which='major', color='lightgrey', linestyle='-')
        ax.xaxis.set_ticks_position("top")
        ax.xaxis.set_label_position("top")
        ax.spines["top"].set_position(("axes", 1.01))
    for ax in [ax1]:
        for depth, colour in zip(formations.values(), zone_colours):
            ax.axhspan(depth[0], depth[1], color=colour, alpha=0.5)
    for ax in [ ax2, ax3]:
        plt.setp(ax.get_yticklabels(), visible = False)
    for label, formation_mid in zip(formations_dict.keys(),
                                    formation_midpoints):
        ax1.text(5, formation_mid, label, rotation=0,
                verticalalignment='center', fontweight='bold',
                fontsize='large')
    plt.tight_layout()
    fig.subplots_adjust(wspace = 0.22)
    fig.savefig("total.tiff", dpi=300)

In [ ]:
multi_window = WindowGenerator(input_width=8,
                               label_width=1,
                               shift=1,
                               train_df = test,
                               test_df = test,
                               val_df = test,
                               label_columns=['CGR', 'SGR'])

In [ ]:
facies_window = WindowGenerator(input_width=8,
                               label_width=1,
                               shift=1,
                                train_df = test,
                               test_df = test,
                               val_df = test,
                               label_columns=['Facies'])

In [ ]:
facies_window

In [ ]:
preds = list()
depth = list()

for idx, (inp, tar) in enumerate(multi_window.test):
  out = multi_lstm_model.predict(inp, verbose = 0)
  out = np.squeeze(out)
  preds.extend(out)
  depth.extend(inp[:, 0, 0].numpy())

In [ ]:
preds = np.array(preds)
preds.shape

In [ ]:
pred_df = data.copy(deep = True)

In [ ]:
pred_df = pred_df.iloc[:-8]

In [ ]:
pred_df.shape

In [ ]:
idx = pred_df[pred_df.Depth > 2779.9284].index.to_numpy()
len(idx)

In [ ]:
pred_df.head()

In [ ]:
pred_df[pred_df.Depth > 2779.9284].head()

In [ ]:
pred_df['LITHOLOGY'] = np.round(pred_df['LITHOLOGY'])

In [ ]:
pred_df['LITHOLOGY'].value_counts()

In [ ]:
data.shape, pred_df.shape

In [ ]:
pred_df.rename(columns={'LITHOLOGY':'val LITHOLOGY'}, inplace=True)
pred_df.rename(columns={'CGR':'val CGR'}, inplace=True)
pred_df.rename(columns={'SGR':'val SGR'}, inplace=True)
pred_df.head()

In [ ]:
df = data.merge(pred_df, on='Depth', suffixes = (None,'_1'))
df.head()

In [ ]:
pd.DataFrame(df).to_csv('dfn.csv')

In [ ]:
df = 'dfn.csv'

In [ ]:
df = pd.read_csv(df)

In [ ]:
makeplot(df, 2350, 3398, formations)

In [ ]:
formations_dict= {}
with open('Formation11_h.csv', 'r') as file:
    next(file)
    for row in csv.DictReader(file, fieldnames=['Formation', 'Top', 'Bottom']):
        formations_dict[row['Formation']]=[float(row['Top']), float(row['Bottom'])]

In [ ]:
formations_dict

In [ ]:
formations_dict['Sarvak'][0]

In [ ]:
formation_midpoints = []
for key, value in formations_dict.items():
    formation_midpoints.append(value[0] + (value[1]-value[0])/2)
formation_midpoints

In [ ]:
zone_colours = [
       '#D8D8D8','#585858','#F7FE2E','#A9E2F3', '#00FFFF']
def makeplot(well, top_depth, bottom_depth, formations):
    fig, ax = plt.subplots(figsize=(3.8, 8))
    ax1 = plt.subplot2grid((1, 3), (0, 0), rowspan=1, colspan = 1)
    ax12 = ax1.twiny()
    ax2 = plt.subplot2grid((1, 3), (0, 1), rowspan=1, colspan = 1, sharey = ax1)
    ax3 = plt.subplot2grid((1, 3), (0, 2), rowspan=1, colspan = 1, sharey = ax1)
    ax10 = ax1.twiny()
    ax10.xaxis.set_visible(False)
    ax1.plot(well["val CGR"], well['Depth'], color = "#088A08", linewidth = 0.5)
    ax1.set_xlabel("S-GR (GAPI)")
    ax1.xaxis.label.set_color("#088A08")
    ax1.set_xlim(0, 150)
    ax1.set_ylabel("TVD (m)")
    ax1.tick_params(axis='x', colors="#088A08")
    ax1.spines["top"].set_edgecolor("#088A08")
    ax1.title.set_color('#088A08')
    ax1.set_xticks([0, 75, 150])
    left_col_value = 0
    right_col_value = 150
    span = abs(left_col_value - right_col_value)
    cmap = plt.get_cmap('hot_r')
    color_index = np.arange(left_col_value, right_col_value, span / 100)
    for index in sorted(color_index):
        index_value = (index - left_col_value)/span
        color = cmap(index_value) 
    ax12.plot(well["CGR"], well['Depth'], color = "#FF0000", linewidth = 0.5)
    ax12.set_xlabel('GR (GAPI)')
    ax12.xaxis.label.set_color("#FF0000")
    ax12.set_xlim(0, 150)
    ax12.tick_params(axis='x', colors="#FF0000")
    ax12.spines["top"].set_position(("axes", 1.075))
    ax12.spines["top"].set_visible(True)
    ax12.spines["top"].set_edgecolor("#FF0000")
    ax12.set_xticks([0, 75, 150])
    ax2.plot(well["LITHOLOGY"], well['Depth'], color = "black", linewidth = 0.5)
    ax2.set_xlabel("GML")
    ax2.set_xlim(0, 1)
    ax2.xaxis.label.set_color("black")
    ax2.tick_params(axis='x', colors="black")
    ax2.spines["top"].set_edgecolor("black")

    for key in lithology_numbers.keys():
        color = lithology_numbers[key]['color']
        hatch = lithology_numbers[key]['hatch']
        ax2.fill_betweenx(well['Depth'], 0, well['LITHOLOGY'], where=(well['LITHOLOGY']==key),
                         facecolor=color, hatch=hatch)
    ax2.set_xticks([])
    ax3.plot(well["val LITHOLOGY"], well['Depth'], color = "black", linewidth = 0.5)
    ax3.set_xlabel("S-GML")
    ax3.set_xlim(0, 1)
    ax3.xaxis.label.set_color("black")
    ax3.tick_params(axis='x', colors="black")
    ax3.spines["top"].set_edgecolor("black")

    for key in lithology_numbers.keys():
        color = lithology_numbers[key]['color']
        hatch = lithology_numbers[key]['hatch']
        ax3.fill_betweenx(well['Depth'], 0, well['val LITHOLOGY'], where=(well['val LITHOLOGY']==key),
                         facecolor=color, hatch=hatch)
    ax3.set_xticks([])
    for ax in [ax1, ax2, ax3]:
        ax.set_ylim(bottom_depth, top_depth)
        ax.grid(which='major', color='lightgrey', linestyle='-')
        ax.xaxis.set_ticks_position("top")
        ax.xaxis.set_label_position("top")
        ax.spines["top"].set_position(("axes", 1.01))
    for ax in [ax1]:
        for depth, colour in zip(formations.values(), zone_colours):
            ax.axhspan(depth[0], depth[1], color=colour, alpha=0.5)
    for ax in [ ax2, ax3]:
        plt.setp(ax.get_yticklabels(), visible = False)
    for label, formation_mid in zip(formations_dict.keys(),
                                    formation_midpoints):
        ax1.text(5, formation_mid, label, rotation=0,
                verticalalignment='center', fontweight='bold',
                fontsize='large')
    plt.tight_layout()
    fig.subplots_adjust(wspace = 0.22)
    fig.savefig("2800-3000.tiff", dpi=300)


In [ ]:
formations = {"Sarvak": [2800.0, 3000.0],
              "Gurpi":[2350.618, 2659.99], 
              "Ilam": [2660.142, 2757.938],
              "Lafan": [2758.135, 2768.956]}

In [ ]:
formations_dict= {}

In [ ]:
with open('Sarvak.csv', 'r') as file:
    next(file)
    for row in csv.DictReader(file, fieldnames=['Formation', 'Top', 'Bottom']):
        formations_dict[row['Formation']]=[float(row['Top']), float(row['Bottom'])]
formations_dict

In [ ]:
makeplot(df, 2800, 3000,formations)

In [ ]:
formations_dict= {}
with open('Formation12_h.csv', 'r') as file:
    next(file) 
    for row in csv.DictReader(file, fieldnames=['Formation', 'Top', 'Bottom']):
        formations_dict[row['Formation']]=[float(row['Top']), float(row['Bottom'])]

In [ ]:
formations_dict

In [ ]:
formations_dict['Sarvak'][0]

In [ ]:
formation_midpoints = []
for key, value in formations_dict.items():
    formation_midpoints.append(value[0] + (value[1]-value[0])/2)  
formation_midpoints
zone_colours = ['#A9E2F3',
       '#585858','#F7FE2E','#D8D8D8', '#00FFFF']

In [ ]:
zone_colours = [
       '#D8D8D8','#585858','#F7FE2E','#A9E2F3', '#00FFFF']
def makeplot(well, top_depth, bottom_depth, formations):
    fig, ax = plt.subplots(figsize=(3.8, 8))
    ax1 = plt.subplot2grid((1, 3), (0, 0), rowspan=1, colspan = 1)
    ax12 = ax1.twiny()
    ax2 = plt.subplot2grid((1, 3), (0, 1), rowspan=1, colspan = 1, sharey = ax1)
    ax3 = plt.subplot2grid((1, 3), (0, 2), rowspan=1, colspan = 1, sharey = ax1)
    ax10 = ax1.twiny()
    ax10.xaxis.set_visible(False)
    ax1.plot(well["val CGR"], well['Depth'], color = "#088A08", linewidth = 0.5)
    ax1.set_xlabel("S-GR (GAPI)")
    ax1.xaxis.label.set_color("#088A08")
    ax1.set_xlim(0, 150)
    ax1.set_ylabel("TVD (m)")
    ax1.tick_params(axis='x', colors="#088A08")
    ax1.spines["top"].set_edgecolor("#088A08")
    ax1.title.set_color('#088A08')
    ax1.set_xticks([0, 75, 150])
    left_col_value = 0
    right_col_value = 150
    span = abs(left_col_value - right_col_value)
    cmap = plt.get_cmap('hot_r')
    color_index = np.arange(left_col_value, right_col_value, span / 100)
    for index in sorted(color_index):
        index_value = (index - left_col_value)/span
        color = cmap(index_value)
    ax12.plot(well["CGR"], well['Depth'], color = "#FF0000", linewidth = 0.5)
    ax12.set_xlabel('GR (GAPI)')
    ax12.xaxis.label.set_color("#FF0000")
    ax12.set_xlim(0, 150)
    ax12.tick_params(axis='x', colors="#FF0000")
    ax12.spines["top"].set_position(("axes", 1.075))
    ax12.spines["top"].set_visible(True)
    ax12.spines["top"].set_edgecolor("#FF0000")
    ax12.set_xticks([0, 75, 150])
    ax2.plot(well["LITHOLOGY"], well['Depth'], color = "black", linewidth = 0.5)
    ax2.set_xlabel("GML")
    ax2.set_xlim(0, 1)
    ax2.xaxis.label.set_color("black")
    ax2.tick_params(axis='x', colors="black")
    ax2.spines["top"].set_edgecolor("black")
    for key in lithology_numbers.keys():
        color = lithology_numbers[key]['color']
        hatch = lithology_numbers[key]['hatch']
        ax2.fill_betweenx(well['Depth'], 0, well['LITHOLOGY'], where=(well['LITHOLOGY']==key),
                         facecolor=color, hatch=hatch)
    ax2.set_xticks([])
    ax3.plot(well["val LITHOLOGY"], well['Depth'], color = "black", linewidth = 0.5)
    ax3.set_xlabel("S-GML")
    ax3.set_xlim(0, 1)
    ax3.xaxis.label.set_color("black")
    ax3.tick_params(axis='x', colors="black")
    ax3.spines["top"].set_edgecolor("black")
    for key in lithology_numbers.keys():
        color = lithology_numbers[key]['color']
        hatch = lithology_numbers[key]['hatch']
        ax3.fill_betweenx(well['Depth'], 0, well['val LITHOLOGY'], where=(well['val LITHOLOGY']==key),
                         facecolor=color, hatch=hatch)
    ax3.set_xticks([])
    for ax in [ax1, ax2, ax3]:
        ax.set_ylim(bottom_depth, top_depth)
        ax.grid(which='major', color='lightgrey', linestyle='-')
        ax.xaxis.set_ticks_position("top")
        ax.xaxis.set_label_position("top")
        ax.spines["top"].set_position(("axes", 1.01))
    for ax in [ax1]:
        for depth, colour in zip(formations.values(), zone_colours):
            ax.axhspan(depth[0], depth[1], color=colour, alpha=0.5)
    for ax in [ ax2, ax3]:
        plt.setp(ax.get_yticklabels(), visible = False)
    for label, formation_mid in zip(formations_dict.keys(),
                                    formation_midpoints):
        ax1.text(5, formation_mid, label, rotation=0,
                verticalalignment='center', fontweight='bold',
                fontsize='large')
    plt.tight_layout()
    fig.subplots_adjust(wspace = 0.22)
    fig.savefig("3000-3200.tiff", dpi=300)

In [ ]:
formations = {"Sarvak": [3000.0, 3200.0],
              "Gurpi":[2350.618, 2659.99], 
              "Ilam": [2660.142, 2757.938],
              "Lafan": [2758.135, 2768.956]}

In [ ]:
formations_dict= {}

In [ ]:
with open('Sarvak_1.csv', 'r') as file:
    next(file) #skip header row
    for row in csv.DictReader(file, fieldnames=['Formation', 'Top', 'Bottom']):
        formations_dict[row['Formation']]=[float(row['Top']), float(row['Bottom'])]
formations_dict

In [ ]:
makeplot(df, 3000, 3200,formations)

In [ ]:
formations_dict= {}
with open('Formation13_h.csv', 'r') as file:
    next(file)
    for row in csv.DictReader(file, fieldnames=['Formation', 'Top', 'Bottom']):
        formations_dict[row['Formation']]=[float(row['Top']), float(row['Bottom'])]

In [ ]:
formations_dict

In [ ]:
formations_dict['Sarvak'][0]

In [ ]:
formation_midpoints = []
for key, value in formations_dict.items():
    formation_midpoints.append(value[0] + (value[1]-value[0])/2)

In [ ]:
formation_midpoints

In [ ]:
zone_colours = [
       '#D8D8D8','#585858','#F7FE2E','#A9E2F3', '#00FFFF']
def makeplot(well, top_depth, bottom_depth, formations):
    fig, ax = plt.subplots(figsize=(3.8, 8))
    ax1 = plt.subplot2grid((1, 3), (0, 0), rowspan=1, colspan = 1)
    ax12 = ax1.twiny()
    ax2 = plt.subplot2grid((1, 3), (0, 1), rowspan=1, colspan = 1, sharey = ax1)
    ax3 = plt.subplot2grid((1, 3), (0, 2), rowspan=1, colspan = 1, sharey = ax1)
    ax10 = ax1.twiny()
    ax10.xaxis.set_visible(False)
    ax1.plot(well["val CGR"], well['Depth'], color = "#088A08", linewidth = 0.5)
    ax1.set_xlabel("S-GR (GAPI)")
    ax1.xaxis.label.set_color("#088A08")
    ax1.set_xlim(0, 150)
    ax1.set_ylabel("TVD (m)")
    ax1.tick_params(axis='x', colors="#088A08")
    ax1.spines["top"].set_edgecolor("#088A08")
    ax1.title.set_color('#088A08')
    ax1.set_xticks([0, 75, 150])
    left_col_value = 0
    right_col_value = 150
    span = abs(left_col_value - right_col_value)
    cmap = plt.get_cmap('hot_r')
    color_index = np.arange(left_col_value, right_col_value, span / 100)
    for index in sorted(color_index):
        index_value = (index - left_col_value)/span
        color = cmap(index_value) 
    ax12.plot(well["CGR"], well['Depth'], color = "#FF0000", linewidth = 0.5)
    ax12.set_xlabel('GR (GAPI)')
    ax12.xaxis.label.set_color("#FF0000")
    ax12.set_xlim(0, 150)
    ax12.tick_params(axis='x', colors="#FF0000")
    ax12.spines["top"].set_position(("axes", 1.075))
    ax12.spines["top"].set_visible(True)
    ax12.spines["top"].set_edgecolor("#FF0000")
    ax12.set_xticks([0, 75, 150])
    ax2.plot(well["LITHOLOGY"], well['Depth'], color = "black", linewidth = 0.5)
    ax2.set_xlabel("GML")
    ax2.set_xlim(0, 1)
    ax2.xaxis.label.set_color("black")
    ax2.tick_params(axis='x', colors="black")
    ax2.spines["top"].set_edgecolor("black")
    for key in lithology_numbers.keys():
        color = lithology_numbers[key]['color']
        hatch = lithology_numbers[key]['hatch']
        ax2.fill_betweenx(well['Depth'], 0, well['LITHOLOGY'], where=(well['LITHOLOGY']==key),
                         facecolor=color, hatch=hatch)
    ax2.set_xticks([])
    ax3.plot(well["val LITHOLOGY"], well['Depth'], color = "black", linewidth = 0.5)
    ax3.set_xlabel("S-GML")
    ax3.set_xlim(0, 1)
    ax3.xaxis.label.set_color("black")
    ax3.tick_params(axis='x', colors="black")
    ax3.spines["top"].set_edgecolor("black")
    for key in lithology_numbers.keys():
        color = lithology_numbers[key]['color']
        hatch = lithology_numbers[key]['hatch']
        ax3.fill_betweenx(well['Depth'], 0, well['val LITHOLOGY'], where=(well['val LITHOLOGY']==key),
                         facecolor=color, hatch=hatch)
    ax3.set_xticks([])
    for ax in [ax1, ax2, ax3]:
        ax.set_ylim(bottom_depth, top_depth)
        ax.grid(which='major', color='lightgrey', linestyle='-')
        ax.xaxis.set_ticks_position("top")
        ax.xaxis.set_label_position("top")
        ax.spines["top"].set_position(("axes", 1.01))
    for ax in [ax1]:
        for depth, colour in zip(formations.values(), zone_colours):
            ax.axhspan(depth[0], depth[1], color=colour, alpha=0.5)
    for ax in [ ax2, ax3]:
        plt.setp(ax.get_yticklabels(), visible = False)
    for label, formation_mid in zip(formations_dict.keys(),
                                    formation_midpoints):
        ax1.text(5, formation_mid, label, rotation=0,
                verticalalignment='center', fontweight='bold',
                fontsize='large')
    plt.tight_layout()
    fig.subplots_adjust(wspace = 0.22)
    fig.savefig("3200-3398.tiff", dpi=300)


In [ ]:
formations = {"Sarvak": [3200.0, 3398.0],
              "Gurpi":[2350.618, 2659.99], 
              "Ilam": [2660.142, 2757.938],
              "Lafan": [2758.135, 2768.956]}

In [ ]:
formations_dict= {}

In [ ]:
with open('Sarvak_2.csv', 'r') as file:
    next(file) 
    for row in csv.DictReader(file, fieldnames=['Formation', 'Top', 'Bottom']):
        formations_dict[row['Formation']]=[float(row['Top']), float(row['Bottom'])]

In [ ]:
formations_dict

In [ ]:
makeplot(df, 3200, 3398,formations)

In [ ]:
framerate = 4410
play_time_seconds = 3
t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(2*np.pi*300*t) + np.sin(2*np.pi*240*t)
Audio(audio_data, rate=framerate, autoplay=True)